In [20]:
import numpy as np
import torch
import torch.nn as nn
import art.attacks.evasion as toolbox
from art.estimators.classification import PyTorchClassifier
# from kd_export import LightNN
from mvtec import test_dataset

In [4]:
class LightNN(nn.Module):
    def __init__(self, num_classes=10):
        super(LightNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(16, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Linear(16 * 56 * 56, 256),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

In [5]:
print("Check current device: ")
# Check if GPU is available, and if not, use the CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available(): # Should return True 
    print(f"Using GPU: {torch.cuda.get_device_name(0)}") # Should show your GPU name
else:
    print("Using CPU")

Check current device: 
Using GPU: NVIDIA GeForce RTX 4060


In [17]:
# epsilons = [0, .05, .1, .15, .2, .25, .3, .35, .4, .45, .5, .55, .6, .65, .7, .75, .8, .85, .9, .95]
# Create the list based on the given parameters
eps_res = 32
eps_min = 1e-9  # Correcting the spacing issue in the input
eps_max = 1.0

# Generating the list of values
eps_list = [eps_min + i * (eps_max - eps_min) / (eps_res - 1) for i in range(eps_res)]
print("Using epsilons:", eps_list)

pretrained_model = "student_model.pth"
pretrained_model_KD = "student_model_KD.pth"
use_cuda=True

# Initialize the network
torch.manual_seed(42)
model = LightNN(num_classes=15).to(device)

# Load the pretrained model
model.load_state_dict(torch.load(pretrained_model, map_location=device, weights_only=True))

# Set the model in evaluation mode. In this case this is for the Dropout layers
model.eval()

Using epsilons: [1e-09, 0.03225806548387097, 0.06451612996774193, 0.09677419445161291, 0.12903225893548387, 0.16129032341935484, 0.19354838790322582, 0.2258064523870968, 0.25806451687096776, 0.2903225813548388, 0.3225806458387097, 0.3548387103225807, 0.38709677480645166, 0.41935483929032263, 0.4516129037741936, 0.4838709682580646, 0.5161290327419354, 0.5483870972258065, 0.5806451617096775, 0.6129032261935484, 0.6451612906774193, 0.6774193551612904, 0.7096774196451613, 0.7419354841290323, 0.7741935486129032, 0.8064516130967743, 0.8387096775806452, 0.8709677420645161, 0.9032258065483871, 0.935483871032258, 0.9677419355161291, 1.0]


LightNN(
  (features): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Linear(in_features=50176, out_features=256, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.1, inplace=False)
    (3): Linear(in_features=256, out_features=15, bias=True)
  )
)

In [21]:
# Define loss function and optimizer
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Wrap the model with PyTorchClassifier
classifier = PyTorchClassifier(
    model=model,
    loss=loss_fn,
    optimizer=optimizer,
    input_shape=(3, 224, 224),  # Adjust to your input shape
    nb_classes=15              # Number of output classes
)

In [18]:
print(test_dataset)
test_images = np.array([data[0].numpy() for data in test_dataset])
test_labels = np.array([data[1] for data in test_dataset])

Dataset FixedImageFolder
    Number of datapoints: 467
    Root location: ./mvtec_dataset\organized_test
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )


In [25]:
# Create FGSM attack
for eps in eps_list:
    attack = toolbox.FastGradientMethod(estimator=classifier, eps=eps)

    # Generate adversarial examples
    x_test_adv_fgsm = attack.generate(x=test_images)

    # Evaluate accuracy on adversarial examples
    predictions = classifier.predict(x_test_adv_fgsm)
    accuracy = np.sum(np.argmax(predictions, axis=1) == test_labels) / len(test_labels)
    print(f"Accuracy on FSGM attack with epsilon {eps}: {accuracy * 100:.2f}%")

Accuracy on FSGM attack with epsilon 1e-09: 100.00%
Accuracy on FSGM attack with epsilon 0.03225806548387097: 88.44%
Accuracy on FSGM attack with epsilon 0.06451612996774193: 78.37%
Accuracy on FSGM attack with epsilon 0.09677419445161291: 70.66%
Accuracy on FSGM attack with epsilon 0.12903225893548387: 67.24%
Accuracy on FSGM attack with epsilon 0.16129032341935484: 60.81%
Accuracy on FSGM attack with epsilon 0.19354838790322582: 56.53%
Accuracy on FSGM attack with epsilon 0.2258064523870968: 52.68%
Accuracy on FSGM attack with epsilon 0.25806451687096776: 49.68%
Accuracy on FSGM attack with epsilon 0.2903225813548388: 48.18%
Accuracy on FSGM attack with epsilon 0.3225806458387097: 46.47%
Accuracy on FSGM attack with epsilon 0.3548387103225807: 44.11%
Accuracy on FSGM attack with epsilon 0.38709677480645166: 43.04%
Accuracy on FSGM attack with epsilon 0.41935483929032263: 42.18%
Accuracy on FSGM attack with epsilon 0.4516129037741936: 40.47%
Accuracy on FSGM attack with epsilon 0.4838

In [26]:
# Create PGD attack
for eps in eps_list:
    pgd_attack = toolbox.ProjectedGradientDescentPyTorch(estimator=classifier, eps=eps)

    # Generate adversarial examples
    x_test_adv_pgd = pgd_attack.generate(x=test_images)

    # Evaluate accuracy on adversarial examples
    predictions = classifier.predict(x_test_adv_pgd)
    accuracy = np.sum(np.argmax(predictions, axis=1) == test_labels) / len(test_labels)
    print(f"Accuracy on PGD attack with epsilon {eps}: {accuracy * 100:.2f}%")

Accuracy on PGD attack with epsilon 1e-09: 100.00%


Accuracy on PGD attack with epsilon 0.03225806548387097: 89.72%


Accuracy on PGD attack with epsilon 0.06451612996774193: 73.66%


Accuracy on PGD attack with epsilon 0.09677419445161291: 43.04%


Accuracy on PGD attack with epsilon 0.12903225893548387: 28.27%


...

Accuracy on PGD attack with epsilon 0.8709677420645161: 0.00%


Accuracy on PGD attack with epsilon 0.9032258065483871: 0.00%


Accuracy on PGD attack with epsilon 0.935483871032258: 0.00%


Accuracy on PGD attack with epsilon 0.9677419355161291: 0.00%


Accuracy on PGD attack with epsilon 1.0: 0.00%


In [ ]:
# Create C&W attack
cw2_attack = toolbox.CarliniL2Method(classifier=classifier, confidence=0.0, max_iter=40, learning_rate=0.01)

# Generate adversarial examples
x_test_adv_cw2 = cw2_attack.generate(x=test_images)

# Evaluate accuracy on adversarial examples
predictions = classifier.predict(x_test_adv_cw2)
accuracy = np.sum(np.argmax(predictions, axis=1) == test_labels) / len(test_labels)
print(f"Accuracy on CW2 attack: {accuracy * 100:.2f}%")

In [ ]:
# Create Universal Perturbations attack
up_attack = toolbox.UniversalPerturbation(classifier=classifier, attacker="deepfool", eps=0.1)

# Generate adversarial examples
x_test_adv_up = up_attack.generate(x=test_images)

# Evaluate accuracy on adversarial examples
predictions = classifier.predict(x_test_adv_up)
accuracy = np.sum(np.argmax(predictions, axis=1) == test_labels) / len(test_labels)
print(f"Accuracy on UP attack: {accuracy * 100:.2f}%")